# Classification on Custom Sensor Dataset

This notebook demonstrates how to:
- Load data from custom files: "dataset.txt", "features", and "targets"
- Perform exploratory data analysis and preprocessing
- Scale features and split data for training/testing
 - Train and evaluate four classifiers: SVM, Logistic Regression, Random Forest, and kNN

**Files Expected:**
 - dataset.txt: Contains the sensor readings, whitespace-delimited.
 - features: Contains feature indices and names (e.g., "1 tBodyAcc-mean()-X").
 - targets: Contains one target label per row.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")

## 1. Data Loading

Read the feature matrix, feature names, and target labels from the provided files.


In [7]:
import pandas as pd

# File paths for the dataset
dataset_file = 'dataset.txt'
features_file = 'features.txt'
targets_file = 'targets.txt'

# Load the feature data (assuming whitespace-delimited, no header)
X = pd.read_csv(dataset_file, delim_whitespace=True, header=None)
print("Dataset shape:", X.shape)

# Load feature names: assuming each line is like "1 tBodyAcc-mean()-X"
features_df = pd.read_csv(features_file, header=None, names=['index', 'feature'])
feature_names = features_df['feature'].tolist()

# Assign the feature names to X's columns (if they match in number)
if len(feature_names) == X.shape[1]:
    X.columns = feature_names
else:
    print("Warning: Number of feature names does not match the number of columns in dataset.")


# Load targets (labels)
y = pd.read_csv(targets_file, header=None, names=['target'])
print("Targets shape:", y.shape)

C:\Users\dvlas\AppData\Local\Temp\ipykernel_8544\3321124904.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X = pd.read_csv(dataset_file, delim_whitespace=True, header=None)


Dataset shape: (10299, 561)
Targets shape: (10299, 1)
